## Phase 1

### Header
#### Course Name: CSE487
#### phase #: 1
#### term project name: Core Trends Analysis


### Q Write down the following information: 
#### 1. Identify problems prevalent in public application domains. The issue(s) you are trying to address: This will be a 100-word abstract in the form of a problem statement.: 
    
Our goal is to research and find correlations between modern day technology users and their demographics. Our project focuses on understanding the current patterns that different groups tend to follow. This is an important issue because it shows what interests the younger and older generations are looking towards. This will help companies decide what they should focus on building towards because groups tend to follow trends. If people follow trends and companies develop products and ideas of those trends they will be able to exponentially grow their companie with their product and ideas.

Data Sources: https://www.pewresearch.org/

#### 2. Name of your data: Core Trends Analysis

#### 3. Background:
   + Source: https://www.pewresearch.org/
   + Time Period: 2011-2019
   + Scope (if there is please specified): Americans
   + Data Unit: Smartphone owners


#### 4 Introduction of dataset
Answer 2 questions:
+ How many columns? Show all the column labels. ((You can use pandas.DataFrame.columns to easily show all the labels of columns.)
    
    There are 74 columns in the dataframe
+ What are they refering to respectively? please write in table form (markdown syntax for drawing tables : [markdown tutorial with examples](https://guides.github.com/features/mastering-markdown/))

In [11]:
import pandas as pd

columns = pd.read_csv(r'C:\Users\Ricky\Desktop\CSE487\CSE487Stocks\data\data1.csv')
print(columns)
print("===Column headers below =========================================================================================================================")
print(columns.keys())

      respid  sample  comp  int_date  lang  cregion  state  density  sfips  \
0          4       1     1    190108     1        3     37        2  37069   
1          7       1     1    190108     1        2     39        3  39025   
2          9       1     1    190108     1        1     34        4  34025   
3         10       1     1    190108     1        2     18        3  18039   
4         11       1     1    190114     1        2     26        2  26027   
...      ...     ...   ...       ...   ...      ...    ...      ...    ...   
1497  300194       2     1    190203     1        4      6        2   6085   
1498  300195       2     1    190203     1        4     53        1  53025   
1499  300196       2     1    190204     1        1     36        5  36061   
1500  300199       2     1    190204     1        4      6        2   6083   
1501  300200       2     1    190205     1        3     48        2  48027   

     usr  ... inc  party  partyln  hh1 hh3  ql1  ql1a qc1    we

In [8]:
#### 5 Hypothesis and conclusion
What hypothesis or conclusion do you want to study from the data?  (Remember this relative to your future phases of task)


SyntaxError: invalid syntax (<ipython-input-8-29c69476e201>, line 2)

#### 6 Write codes to load the data 
(recommend: pandas package: 10 mins tutorial: [10 mins tutorial](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#min) )

#### 7 IDA (Initial Data Analysis) 
[reference to do IDA](https://www.sciencedirect.com/science/article/pii/S0022522315017948)  \\
Please specify what you've done (text and code) for each operation (at least 10 operations). If some steps will produce a picture, please show this picture


##### 1

In [ ]:
# code

#### 2

In [ ]:
# code


#### 3

In [ ]:
# code

#### 4

In [ ]:
# code

#### 5

In [ ]:
# code

#### 6

In [ ]:
# code

#### 7

In [ ]:
# code

#### 8

#### 9

In [ ]:
# code

In [ ]:
# code

#### 10

In [ ]:
# code

In [ ]:
##### more

8. (option)Remeber to save your dataSet after processing for future usage:


### Submission guideline:
1. write your code and report in this notebook. 
2. save this notebook as <person1ubname_person2ubname.ipynb>.
3. save your raw data in data folder.
4. save the notebook file with data folder in a folder named <Jiayi_person1ubname_person2ubname.zip> or <Ping_person1ubname_person2ubname.zip> or <Chen_person1ubname_person2ubname.zip> depended on which TA you have choosed. 
5. upload your zip file to timberlake server.

### Grading:
    0 – no submission by deadline
    15 – completed submission with all the requirements
    10 – there are missing items, or unsatisfactory in some respects
    5 – submission present but many required items missing or code not working
    
Once the deadline is past, you get a zero for the phase if you did not submit anything, even though you need to complete the phase to work on the next phase.


### Example Answer
(I am using the dataset from https://www.kaggle.com/gpreda/covid-world-vaccination-progress to provide a sample answer )

1. Source of Data: https://ourworldindata.org/
2. Give a name to your dataset: COVID-19 WVP (World Vaccination Progress)
3. Background:
   + Data were collected by Our World in Data
   + Time Period: 2020-2021
   + Scope (if there is please specified): Global
   + Data Unit: Individual country


|Column label|description|
|-|-|
| Country | this is the country for which the vaccination information is provided |
| Country ISO Code | ISO code for the country|
|Date | date for the data entry; for some of the dates we have only the daily vaccinations, for others, only the (cumulative) total|
|Total number of vaccinations| this is the absolute number of total immunizations in the country |
|Total number of people vaccinated| a person, depending on the immunization scheme, will receive one or more (typically 2) vaccines; at a certain moment, the number of vaccination might be larger than the number of people|
|Total number of people fully vaccinated| the number of people that received the entire set of immunization according to the immunization scheme (typically 2); at a certain moment in time, there might be a certain number of people that received one vaccine and another number (smaller) of people that received all vaccines in the scheme|
|Daily vaccinations (raw) | for a certain data entry, the number of vaccination for that date/country|
|Daily vaccinations | for a certain data entry, the number of vaccination for that date/country|
|Total vaccinations per hundred| ratio (in percent) between vaccination number and total population up to the date in the country|
|Total number of people vaccinated per hundred | ratio (in percent) between population immunized and total population up to the date in the country|
|Total number of people fully vaccinated per hundred |ratio (in percent) between population fully immunized and total population up to the date in the country|
|Number of vaccinations per day | number of daily vaccination for that day and country|
|Daily vaccinations per million | ratio (in ppm) between vaccination number and total population for the current date in the country|
|Vaccines used in the country | total number of vaccines used in the country (up to date)|
|Source name | source of the information (national authority, international organization, local organization etc.)|
|Source website | website of the source of information|

4. Introduction of dataset: Answer 2 questions:
+ How many columns? Show all the column labels. ((You can use pandas.DataFrame.columns to easily show all the labels of columns.)
+ What are they refering to respectively? please write in table form (markdown syntax for drawing tables : [markdown tutorial with examples](https://guides.github.com/features/mastering-markdown/))

5. What hypothesis or conclusion do you want to study from the data?  (Remember this relative to your future phases of task)\\
The relation between time with respect to increaseing of vaccinated people per day is decreasing negative exponetially (You don't need to be too specific on the hypothesis, something vague is fine. But the hypothesis mustn't be too obvious to analysis, e.g. find the country with largest vaccinated people ). \\

6. Write codes to load the data (recommend: pandas package) save your data in your google drive "./data" folder

In [ ]:
import pandas as pd
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/My Drive/data/test.csv")

7. IDA (Initial Data Analysis)

Statement: I want to take a look at the brief description of data



```
df.describe()
```


8. (option)Remeber to save your dataSet after processing for future usage:


In [ ]:
df.to_csv(index=False) # for example